# Kaggel Emotion Detection Code
## DATA622 - Machine Learning and Big Data
## Erik Nylander

For the Kaggle Competion project I ran the model as normal but at the end of training I saved out the model in JSON format and saved the model weights. These are then loaded by the following code and used to make predictions on the test set. In the training of the original model I used images that are 64x64 pixels in size so I reshaped the test images to match these. I also needed to convert the images to greyscale to match the original model. Code for the Convolutional Neural Network that was used to classify the data can be found [here](https://github.com/eriknylander99/DATA622/blob/master/emotion_detection/emd_aug_cnn.py). 

In [1]:
# Loading the required packages
import cv2
import os
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.models import model_from_json
from PIL import Image
from keras import backend as K
from keras.optimizers import RMSprop
K.set_image_dim_ordering('th')

Using Theano backend.


### Loading and Compiling the Saved Model

In [2]:
# load the saved model in json format
json_file = open('d:/erik/documents/github/mlprojects/model/aug_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
print("Loaded model from disk")

# load the saved weights into the loaded model
loaded_model.load_weights('d:/erik/documents/github/mlprojects/model/aug_flow.h5')
print("Loaded model weights from disk")
 
# compiling the loaded model with the weights
rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
loaded_model.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])
print("Compiled Model")

Loaded model from disk
Loaded model weights from disk
Compiled Model


### Loading the Data for the Kaggle Competition

In [3]:
# Loads the test images for the kaggle competition.
loc = 'd:/erik/documents/github/facial_expressions/test/'
img_names = os.listdir(loc)
x_test = []
actual = 0
for i in img_names:
    img = Image.open(loc + i).resize((64, 64))
    img = img.convert('L')
    img_arr = list(img.getdata())
    img_matrix = np.array(img_arr).reshape(1, 64, 64)   ## im has 3 layers

    x_test.append(img_matrix)
    actual += 1

x_test = np.reshape(x_test, (actual, 1 , 64, 64))

x_test = x_test.astype('float32')
x_test /= 255

In [4]:
# We have all 263 images.
len(x_test)

263

### Predicting the Classes and Probabilities for the Test Data

In [6]:
# Predicting the classses and probabilites for the test set
classes = loaded_model.predict_classes(x_test)
probs = loaded_model.predict_proba(x_test)

256/263 [============================>.] - ETA: 0s

### Labeling the Classes for Each of the Images

In [10]:
# Create a dictionary of the classes to be used in labeling each image.
emotions = ['neutral', 'anger', 'surprise', 'sadness', 'happiness', 'contempt', 'disgust', 'fear']
i_emotions = [0,1,2,3,4,5,6,7]
e = dict(zip(i_emotions, emotions))

In [11]:
# Create the emotion label and the probability for each label. We only return the highest probability label.
emotion = []
probability = []
for i in range(len(classes)):
    emotion.append(e[classes[i]])
    probability.append(probs[i][classes[i]])

### Creating a DataFrame to Contain the Predictions

In [12]:
# Creates a dataframe with images, emotions, class numbers, and probabilites.
data = {'Image' : img_names,
        'Emotion' : emotion,
        'Class' : classes, 
        'Probability' : probability}
test = pd.DataFrame(data, columns = ['Image', 'Emotion', 'Class', 'Probability'])

In [13]:
test.head()

,Image,Emotion,Class,Probability
0,108a.jpg,neutral,0,0.336228
1,110b.jpg,happiness,4,0.716391
2,118a.jpg,happiness,4,0.673479
3,119a.jpg,happiness,4,0.456203
4,11a.jpg,happiness,4,0.452810


### Writing the Predictions to a CSV File for Submission

In [14]:
test.to_csv('d:/erik/documents/github/mlprojects/model/kaggle3_en.csv', 
            index = False, columns = ['Image', 'Emotion'])